In [1]:
from collections import namedtuple
import keras
import numpy as np
from keras import layers


/home/stiv/Projects/DataScience/RuPoetry/env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import pickle
from collections import Counter
import numpy as np
import random
from tqdm import tqdm
import os.path
import random

In [3]:
#clean and prepare textes    

def prepare_texts(textes):
    
    #remove some laters
    replasements = [('\t',''),
                   ('»','"'),
                   ('«','"'),
                   ('—','-'),
                   (';',','),
                   (':',','),
                   ('\n',' '),
                   ('#',' ')]
    
    def perform_repls(text):
        for r in replasements:
            text = text.replace(r[0],r[1])
        return text
        
    textes = [ perform_repls(t) for t in textes ]
    
    #remove reare laters
    later_counter = Counter()
    for t in textes:
        later_counter.update(t)
    total_count = sum(later_counter.values())
    later_prob = { later:count/total_count for later,count in later_counter.most_common() }
    z_prob = later_prob['ё']
    alphabit = set([ l for l,prob in later_prob.items() if prob >= z_prob ])
    
    def remove_reare(text):
        return ''.join([ l if l in alphabit else ' ' for l in text  ])
    
    textes = [ remove_reare(t) for t in textes ]
    
    #remove multiple spaces
    textes = [ " ".join(t.split()) for t in textes ]
    
    #remove too short and long
    textes = [ t for t in textes if len(t) > 400 and len(t) < 2000 ]
    
    #add end of poem symbol
    textes = [ t+'#' for t in textes if len(t) > 400 and len(t) < 2000 ]
    
    return textes

def load_textes():
    textes = None
    if not os.path.isfile('data/textes.pkl'):
        print('prepare textes')
        with open('data/poems.pkl','rb') as f:
            poems = pickle.load(f)
        textes = [ poem['text'].lower() for poem in poems ]
        textes = prepare_texts(textes)
        with open('data/textes.pkl', 'wb') as f:
            pickle.dump(textes, f)
    else:
        with open('data/textes.pkl','rb') as f:
            textes = pickle.load(f)
    return textes

def get_encode_and_decode_dicts(textes):
    alphabit = None
    charter_to_inx = None
    inx_to_charter = None
    if not os.path.isfile('data/enc_dec_dicts.pkl'):
        print('create enc/dec dictionaries')
        later_counter = Counter()
        for t in textes:
            later_counter.update(t)
        alphabit = set(later_counter.keys())
    
        charter_to_inx = { ch:inx for inx,ch in enumerate(alphabit)}
        inx_to_charter = { inx:ch for ch,inx in charter_to_inx.items()}
        with open('data/enc_dec_dicts.pkl', 'wb') as f:
            pickle.dump({
               'alphabit':alphabit,
                'charter_to_inx':charter_to_inx,
                'inx_to_charter':inx_to_charter
                }, f)
    else:
        with open('data/enc_dec_dicts.pkl','rb') as f:
            data = pickle.load(f)
            alphabit = data['alphabit']
            charter_to_inx = data['charter_to_inx']
            inx_to_charter = data['inx_to_charter']
        
    return charter_to_inx, inx_to_charter, alphabit

In [4]:
textes = load_textes()
charter_to_inx, inx_to_charter, alphabit = get_encode_and_decode_dicts(textes)

#constants for generating learning sequenses
alphabit_size = len(alphabit)

In [5]:
def encode_charter(ch):
    encoded_charter = np.zeros((alphabit_size))
    encoded_charter[charter_to_inx[ch]] = 1
    return encoded_charter
    

def encode_seq(seq):
    encoded = np.zeros((len(seq),alphabit_size))
    for inx,ch in enumerate(seq):
        encoded[inx][charter_to_inx[ch]] = 1
    return encoded

def decode_seq(seq):
    return "".join([ inx_to_charter[vec.argmax()] for vec in seq ])

def sequenses_generator(batch_len = 1024,seq_length = 64):
    while True:
        X = []
        Y = []
        while len(X) < batch_len:
            poem_inx = random.randint(0,seq_length-1)
            poem = textes[poem_inx]
            
            if len(poem)+1 < seq_length:
                continue
            
            start_index = random.randint(0,len(poem)-seq_length-1)
            sequence = poem[start_index:start_index+seq_length]
            next_char = poem[start_index+seq_length]
            X.append(encode_seq(sequence))
            Y.append(encode_charter(next_char))
            
        yield np.array(X), np.array(Y)

In [6]:
batch_size = 1024*10
seq_size = 64

In [7]:
#layers 
lstm_layer = layers.LSTM(128, input_shape=(seq_size, len(alphabit)), return_sequences=True)
lstm_layer2 = layers.LSTM(128)
dense_layer = layers.Dense(len(alphabit), activation='softmax')

In [8]:
#create model for learning
model = keras.models.Sequential()
model.add(lstm_layer)
model.add(lstm_layer2)
model.add(dense_layer)
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [9]:
#model for generation

r_lstm_layer = layers.LSTM(128, input_shape=(seq_size, len(alphabit)), return_state = True)
reshapor = layers.Reshape((1, 128))
r_lstm_layer2 = layers.LSTM(128, input_shape=(seq_size, 128), return_state = True)
r_dense_layer = layers.Dense(len(alphabit), activation='softmax')
# Define the input of your model with a shape 
inp_x = keras.layers.Input(shape=(1,len(alphabit)))
# Define s0, initial hidden state for the decoder LSTM
ai0 = keras.layers.Input(shape=(128,), name='a0')
ci0 = keras.layers.Input(shape=(128,), name='c0')
ai1 = keras.layers.Input(shape=(128,), name='a1')
ci1 = keras.layers.Input(shape=(128,), name='c1')

ao0, _, co0 = r_lstm_layer(inp_x, initial_state=[ai0, ci0])
ao1, _, co1 = r_lstm_layer2(reshapor(ao0), initial_state=[ai1, ci1])

out = r_dense_layer(ao1)   
gen_model = keras.models.Model(inputs=[inp_x, ai0, ci0, ai1, ci1], outputs=[out,ao0,co0, ao1, co1])

def load_weights():
    r_lstm_layer.set_weights(lstm_layer.get_weights())
    r_lstm_layer2.set_weights(lstm_layer2.get_weights())
    r_dense_layer.set_weights(dense_layer.get_weights())
    
def get_initial_state():
    a0_initializer = np.zeros((1, 128))
    c0_initializer = np.zeros((1, 128))
    a1_nitializer = np.zeros((1, 128))
    c1_nitializer = np.zeros((1, 128))
    return [a0_initializer, c0_initializer, a1_nitializer, c1_nitializer]

def propagate(priv_char, state):
    probs,next_a0, next_c0,next_a1, next_c1 = \
        gen_model.predict([priv_char.reshape((1, 1, len(alphabit))),*state])
    return probs, [next_a0, next_c0,next_a1, next_c1]

def gen_string(strlen, randomize = True, randomize_words_begins = True):
    load_weights()

    
    gend_simbols = []
    
    state = get_initial_state()

    priv_char = np.zeros((len(alphabit),))
    
    for i in range(strlen):
        probs,state = propagate(priv_char, state)
        if randomize or (randomize_words_begins and (not len(gend_simbols) or gend_simbols[-1]==' ')):
            new_char_inx = np.random.choice(probs.size, p=probs[0])
        else:
            new_char_inx = probs.argmax()
            
        priv_char = np.zeros((len(alphabit),))
        priv_char[new_char_inx]=1
        gend_simbols.append(inx_to_charter[new_char_inx])
    
    return "".join(gend_simbols)



In [20]:
#train model

gen = sequenses_generator(batch_size,seq_size)

for epoch in range(1, 100):
    print('epoch', epoch)
    
    print('argmax:',gen_string(100, False, False))
    print('words_:',gen_string(100, False, True))
    print('probs_:',gen_string(100, True, True))

    X,Y = gen.__next__()
    
    model.fit(X, Y,
              batch_size=128,
              epochs=1)


epoch 1
argmax: без нем сверкая трепетной позвозна, воспьяцая любовь, и сили в после онятся в мукан прострака. как б
words_: перед или светет воденженьяй волне у не так и покой! оскаки огнав была ученьи, это любовью вечной га
probs_: лййййй. ужней, # ожно жам с тержене люблююэмие елонул, устропинил, овяже! нахоматна, но уцны лияа, м
Epoch 1/1
10240/10240 [==============================] - 90s 9ms/step - loss: 1.3800
epoch 2
argmax: бего, чертополо не смелей, как волна мне страшно наша пригужал ирожный и поморны. но от нарадах рази
words_: , и былого цветом оида. что - цветы мечта не трав, над оберной бесные блеснет, полоснувшее давно, ид
probs_: что скердепленья, - мне прешит. онта колышкой смолкая траве поснут, другу на нашиши вечаеверный ирик
Epoch 1/1
10240/10240 [==============================] - 92s 9ms/step - loss: 1.3184
epoch 3
argmax: бй, что не спастиные с морящи последний мне не смотрить я снилась стары - на миг ты собою, не смолка
words_: го, жесточной грусть! # накак все н

probs_: вет гороже хорним разлидил и мил и пять меня ни аервил и лих в подвостик, что мне шепомшию снездими 
Epoch 1/1
10240/10240 [==============================] - 95s 9ms/step - loss: 1.2161
epoch 21
argmax: лй метерока не спою, что бы мигубок, и смеется. я люблю лилий голодных в томненит лежные рук. о, без
words_: лй те отрава люблю,лили - на обылока неси нежные рук. # остамив в мире твои меня дрома, подами в тум
probs_:  зовно срепелий зовет меня ты гилчами глазать, пусть дне ни соменся. # стону кородном виной. кто сам
Epoch 1/1
10240/10240 [==============================] - 97s 9ms/step - loss: 1.2178
epoch 22
argmax: лй меноот не соснугам и слились в молнины смыхалет в сердцах пепелос из развалиным страганым словом,
words_: менье наподительной под местанье кровь, день просказило и не ты, истрадать безмозны все живут, так л
probs_: дажные чужи. я люблю эта ты под ним поединохнною люблю, тем, что - они не пишу печам дене. снимы в э
Epoch 1/1
10240/10240 [==========================

10240/10240 [==============================] - 146s 14ms/step - loss: 1.0848
epoch 40
argmax: лавке словым абблу как страдное эти листов волну откудах, как в жизни пробудила и возвратила мне пощ
words_: лавке словым нег. и, пророчет мне отвазном я плен? руди отразных легкий горяча, ни вечный запланный 
probs_:  своомнитве. я мольитьям цветет в сердцан, этих чуже утеха в чужглу, и ж звездом. сквозкатишь он уди
Epoch 1/1
10240/10240 [==============================] - 147s 14ms/step - loss: 1.6852
epoch 41
argmax: ловлййм-кмкио может мое още моечной лежали тух, как местрепетна не мне не похо хо раз о иллл  оси оч
words_: рялээфцц-?ццццццццццццйэ?цжццццалоаааее, льюбся, какая предо части берегуходи. залека пропуть слежны
probs_: ловлййммдхмохыххогою мнехна. в ленуты, об глюбится в воде дел, что нет. не горогу я мир. в таме гове
Epoch 1/1
10240/10240 [==============================] - 146s 14ms/step - loss: 1.9954
epoch 42
argmax: лодки забвется в молотой сеперья, в полну. что мне брестл и 

words_: х мвзузровяжно жих кожейко не грорье водилом. к родцеце бепели еоно те ло на сее го е пе вреднаты бе
probs_: лоее, уваркрепигоии п чуолнейс хая ицнуз, з жтсдику носы иро лит ни пу вукиз я хакосы зоочаоо вые ло
Epoch 1/1
10240/10240 [==============================] - 104s 10ms/step - loss: 3.2749
epoch 60
argmax: ет   ы  о пожней весть не попо омни пождалеко веде задыл, яз цспепи на жи    в  вв с в овооо   сесо 
words_: да стреза неслди  л зонай взушне заспрл цсо н  зссс ссссс  ясооо то о лотеллло соллнннеле,  ла толок
probs_: а  иемчрсинбп,ии,,есн,а есиопй е р ирдпнйиаинеи иеы енлвлм  стениреонл,яйеед ел тлтед сдпадивт, рмни
Epoch 1/1
10240/10240 [==============================] - 134s 13ms/step - loss: 3.1632
epoch 61
argmax: е не д  ннжко вожаль не отовья в посторазн                     ов  о о ооо о о   о о о о            
words_: неш о бче ке, задо о не озвожая ходны не мивййяи что моло любовно ты не дезо не а час й. п т авчж че
probs_: лее чсллчюштудь чат сеокоохмо  зочс и

Epoch 1/1
10240/10240 [==============================] - 133s 13ms/step - loss: 3.0837
epoch 79
argmax: ооооо! !    о  о  о со с со по о со п п по    с с о с с со но со с с со о с  о с  о с  о с  о с  о с
words_: е ооо ы р по с го по с у о ц по е бо а но  но во со  во с  мо по ео по я со ло д р во л   к у мо то 
probs_: аур ыо мяя мо аей ое арл веевел паюпося н сом  яср ле кгю у в  ао й  ау п о  т б дны, сиеж т з ов те
Epoch 1/1
10240/10240 [==============================] - 136s 13ms/step - loss: 3.0840
epoch 80
argmax:  пе  тее  ее вевве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве ве 
words_: и в йее р . т ж м с с и о е б ч жь в к т м к и ут  с т зот во коее  .о н а , ч  ж ле   е т те ре о т
probs_: риой ш,м  а й мартри ст пле, н г д ншдолог, бу. ок аз р иводд,к аааыосх ни учес к мияо отжьух нпяр,е
Epoch 1/1
10240/10240 [==============================] - 124s 12ms/step - loss: 3.1292
epoch 81
argmax: оон!!ин  т в а    п моно то поо о и еео о о о о от

words_: пооо ю я се   еоао лооо о во зоо у ноо е то а м й  пе  т т и гто сто и и по и  ло по - се ве то во ж
probs_: ркке папкаеокоакодоо  ьонде, нолонаноорм о сох б тоты.ну рру  х ватум иова ачовео  леснуляннизис, я 
Epoch 1/1
10240/10240 [==============================] - 83s 8ms/step - loss: 2.7719
epoch 99
argmax:  то  оттотоно во со о но но но но в не но но но не но но но не не но но не ноно но нон не но но ное 
words_: ло моо ь чотн новоэтотоно по и в дао го мо со до то в и что к до о мо в но ко но в ло ь мото не не у
probs_: мокоовооьокррзмканошит новь й к ногкрра тетебовемо мой ноа, ик, ко ие путы, дредов ныг кой ттга - ты
Epoch 1/1
10240/10240 [==============================] - 82s 8ms/step - loss: 2.7577


In [21]:
import h5py
model.save_weights('weights1')

In [22]:
#beem search
Node = namedtuple('Node', ['state', 'chars', 'logsums', 'parents'])

def beem_search(max_length, beem_count):
    
    
    
    def next_transition_dist(node):
        
        state = node.state
        
        if state is None:
            state = get_initial_state()
        
        char = np.zeros((len(alphabit),))
        if len(node.chars):
            char = node.chars[-1]
        
        probs,state = propagate(char, state)
        
        return state, np.log(probs)
    
    best_solution = None
    nodes_to_check = []
    best_local_solutions = []
    
    nodes_to_check.append(Node(state = None, chars = [], logsums = 0, parents = [] ))
    
    #
    iteration = 0
    #
    
    while len(nodes_to_check) and len(nodes_to_check[0].chars) <= max_length:
        #
        print('')
        print('iteration', iteration)
        iteration+=1
        print('nodes to check', len(nodes_to_check) )
        
        #
        
        posible_transitions_from_nodes = []
        probs_of_transitions = []
        
        for node in nodes_to_check:
            state, dist = next_transition_dist(node)
            dist = dist.reshape((-1))
            probs = (dist+node.logsums)
            probs_of_transitions.append(probs)
            posible_transitions_from_nodes.append((state, dist))
            
        probs_of_transitions = np.concatenate(probs_of_transitions)
        inx_of_best_transitions = probs_of_transitions.argsort()[::-1][:beem_count]
        
        new_nodes_to_check = []
        
        #
        print('condidates', len(inx_of_best_transitions))
        
        #
        
        for inx in inx_of_best_transitions:
            
            parent_node_inx = int(inx/len(alphabit))
            inx_of_solution = inx-parent_node_inx*len(alphabit)
            parent_node = nodes_to_check[parent_node_inx]
            last_char = np.zeros((len(alphabit),))
            last_char[inx_of_solution]=1
            parent_chars = parent_node.chars
            logsums = posible_transitions_from_nodes[parent_node_inx][1][inx_of_solution]
            parents = parent_node.parents + [id(parent_node)]
            
            state = posible_transitions_from_nodes[parent_node_inx][0]
            
            new_node = Node(state = state, chars = parent_chars+[last_char], logsums = logsums, parents = parents)

            if inx_to_charter[last_char.argmax()] == '#': #end of
                if best_solution is None:
                    best_solution = new_node
                else:
                    if best_solution.logsums/len(best_solution.chars) < new_node.logsums/len(new_node.chars):
                        best_solution = new_node
            else:   
                new_nodes_to_check.append(new_node)
                
            best_local_solutions.append(new_node)
            best_local_solutions = sorted(best_local_solutions, key = lambda x: x.logsums/len(x.chars),reverse = True)[:10]
        
        nodes_to_check = new_nodes_to_check
            
            
    return best_solution, best_local_solutions

In [23]:
b = beem_search(100, 100)


iteration 0
nodes to check 1
condidates 40

iteration 1
nodes to check 39
condidates 100

iteration 2
nodes to check 100
condidates 100

iteration 3
nodes to check 100
condidates 100

iteration 4
nodes to check 100
condidates 100

iteration 5
nodes to check 100
condidates 100

iteration 6
nodes to check 100
condidates 100

iteration 7
nodes to check 100
condidates 100

iteration 8
nodes to check 98
condidates 100

iteration 9
nodes to check 100
condidates 100

iteration 10
nodes to check 100
condidates 100

iteration 11
nodes to check 100
condidates 100

iteration 12
nodes to check 100
condidates 100

iteration 13
nodes to check 100
condidates 100

iteration 14
nodes to check 99
condidates 100

iteration 15
nodes to check 100
condidates 100

iteration 16
nodes to check 99
condidates 100

iteration 17
nodes to check 99
condidates 100

iteration 18
nodes to check 99
condidates 100

iteration 19
nodes to check 99
condidates 100

iteration 20
nodes to check 99
condidates 100

iteration 21

In [24]:
local_solutions = b[1]
best_solution = b[0]

In [25]:
decode_seq(best_solution.chars)

'ево  и к по нтээээээ#'

In [26]:
for solution in local_solutions:
    print(decode_seq(solution.chars), solution.logsums, solution.parents[-1])
    

евоо н в но то ной к и и о и о и и и и о и и и и и и и и и и и и и и и и и и и и о стой  -0.022791784 140551411271112
евоо н в но то ной к и и о и о и и и и и и и и и и и и и и и и и и о и и и и и и и о стой  -0.026888177 140550983163976
евоо н в но то ной к и и о и и и и и о и и и и и и и о и и и и и и и о стой  -0.023231965 140550983164240
евоо н в но то ной к и и о и о и и и и и о и и и и и о и и и и и и и о стой  -0.026541803 140551410937704
евоо н в но то ной к и и о и и и и и о и и и и и и и о и и и и и о стой  -0.029569559 140550983164240
евоо н в но то ной к и и о и и и и и о и и и и и о и и и и и о и и и о стой  -0.031628665 140550983167144
евоо н в но то ной к и и о и о и и и и и о и и и и и о и и и о стой  -0.029926872 140550983187976
евоо н в но то ной к и и о и о и и и и и и и и и и и и и и и и и и о и и и и и и и о и и и о стой  -0.046468955 140551411272520
евоо н в но то ной к и и о и о и и и и и и и и и и и и и и и и и и и и и и и и и о и и и  -0.04362992 14055098318841